In [99]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

In [100]:
season_list = ['1718', '1819', '1920', '2021', '2122', '2223', '2324', '2425']
league_eng = 'E0'


df_list_eng = []

for season in season_list:
    data_url = f'https://www.football-data.co.uk/mmz4281/{season}/{league_eng}.csv'
    df = pd.read_csv(data_url)
    df_list_eng.append(df)

main_df = pd.concat(df_list_eng)

main_df['Date'] = pd.to_datetime(main_df['Date'], format='%d/%m/%Y')
main_df = main_df.sort_values(by='Date')

df = main_df
df = df[::-1]
df.reset_index(inplace=True)

/var/folders/rt/kk5y63zd2dl20zp4x97pdp2m0000gn/T/ipykernel_47617/2438252128.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.reset_index(inplace=True)


In [101]:
df_swe = pd.read_csv('https://www.football-data.co.uk/new/SWE.csv')

df_swe['Date'] = pd.to_datetime(df_swe['Date'], format='%d/%m/%Y')
df_swe= df_swe.sort_values(by='Date')
df_swe = df_swe[::-1]

df_swe.reset_index(inplace=True)

In [102]:
def get_teams_result(df):
    resultat = {}

    teams = set(df["HomeTeam"]).union(set(df["AwayTeam"]))

    for team in teams:
        resultat[team] = {}
        
    for _, rad in df.iterrows():
        hemma, borta = rad["HomeTeam"], rad["AwayTeam"]
        mål_a, mål_b = rad["FTHG"], rad["FTAG"]

        if hemma not in resultat:
            resultat[hemma] = {}
        if borta not in resultat:
            resultat[borta] = {}

        if borta not in resultat[hemma]:
            resultat[hemma][borta] = {"Vinster": 0, "Förluster": 0, "Oavgjorda": 0}
        if hemma not in resultat[borta]:
            resultat[borta][hemma] = {"Vinster": 0, "Förluster": 0, "Oavgjorda": 0}

        if mål_a > mål_b: 
            resultat[hemma][borta]["Vinster"] += 1
            resultat[borta][hemma]["Förluster"] += 1
        elif mål_a < mål_b: 
            resultat[borta][hemma]["Vinster"] += 1
            resultat[hemma][borta]["Förluster"] += 1
        else:
            resultat[hemma][borta]["Oavgjorda"] += 1
            resultat[borta][hemma]["Oavgjorda"] += 1
            
    return resultat

#get_teams_result(df)['Man City']

In [103]:
def get_home_vs_away_summary(df, home_team, away_team):

    matcher = df[(df["HomeTeam"] == home_team) & (df["AwayTeam"] == away_team)]

    resultat = {"Vinster": 0, "Förluster": 0, "Oavgjorda": 0}

    for _, rad in matcher.iterrows():
        mål_hemma = rad["FTHG"]
        mål_borta = rad["FTAG"]

        if mål_hemma > mål_borta:
            resultat["Vinster"] += 1
        elif mål_hemma < mål_borta:
            resultat["Förluster"] += 1
        else:
            resultat["Oavgjorda"] += 1

    return resultat

#get_home_vs_away_summary(df,'Liverpool', 'Arsenal')

In [104]:
def get_team_league_test():
    teams = []

    # Hantera engelska ligor (Div)
    all_eng_teams = df['HomeTeam'].astype(str).tolist() + df['AwayTeam'].astype(str).tolist()   
    unique_eng_teams = np.unique(all_eng_teams)
    teams.extend([str(team) for team in unique_eng_teams])

    # Hämta ligor från båda dataframes
    english_leagues = sorted(df['Div'].astype(str).unique().tolist())
    swedish_leagues = sorted(df_swe['League'].astype(str).unique().tolist())

    all_leagues = english_leagues + swedish_leagues
    return teams, all_leagues

In [105]:
def get_team_league():
    teams_by_league = []
    all_leagues = []

    # Hantera engelska ligor
    english_leagues = sorted(df['Div'].astype(str).unique().tolist())
    for league in english_leagues:
        league_df = df[df['Div'] == league]
        teams = league_df['HomeTeam'].astype(str).tolist() + league_df['AwayTeam'].astype(str).tolist()
        unique_teams = sorted(np.unique(teams))
        teams_by_league.append(unique_teams)
        all_leagues.append(league)

    # Hantera svenska ligor, ta bort whitespaces och duplicates
    swedish_leagues_raw = df_swe['League'].astype(str).str.strip()
    swedish_leagues = sorted(swedish_leagues_raw.unique().tolist())

    for league in swedish_leagues:
        league_df = df_swe[df_swe['League'].str.strip() == league]
        teams = league_df['Home'].astype(str).tolist() + league_df['Away'].astype(str).tolist()
        unique_teams = sorted(np.unique(teams))
        teams_by_league.append(unique_teams)
        all_leagues.append(league)

    return teams_by_league, all_leagues

In [106]:
col_list = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'B365H', 'B365D', 'B365A']

In [107]:
def get_team_form(df, team, num_matches=20):

    team_matches = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)].copy()

    team_matches = team_matches.sort_values(by='Date', ascending=False).head(num_matches)

    form = []

    for _, row in team_matches.iterrows():
        is_home = row['HomeTeam'] == team
        goals_for = row['FTHG'] if is_home else row['FTAG']
        goals_against = row['FTAG'] if is_home else row['FTHG']

        if goals_for > goals_against:
            form.append('W')
        elif goals_for == goals_against:
            form.append('D')
        else:
            form.append('L')

    return form[::-1]  

#get_team_form(df, 'Tottenham')

In [108]:
def get_fixture_by_league(league):
    df_next_games = pd.read_csv('https://www.football-data.co.uk/fixtures.csv')
    
    df_next_games['Date'] = pd.to_datetime(df_next_games['Date'], format='%d/%m/%Y', errors='coerce')
    today = pd.Timestamp.today().normalize()
    df_next_games = df_next_games[df_next_games['Date'] >= today]

    if league in df_next_games['Div'].unique():
        league_fixtures = df_next_games[df_next_games['Div'] == league]
        fixtures_list = league_fixtures[['HomeTeam', 'AwayTeam', 'Date', 'Time']].dropna().to_dict(orient='records')
        return fixtures_list
    return []

In [133]:
def create_league_table(league):
    # Hämta det nuvarande året
    current_year = datetime.now().year
    
    # Kontrollera om dagens datum är före eller efter 1 augusti
    if datetime.now().month < 8:
        start_date = f"{current_year - 1}-08-01"  # Om innan 1 augusti, börja från förra året
        end_date = f"{current_year}-07-01"        # Slutdatum är 1 juli i år
    else:
        start_date = f"{current_year}-08-01"      # Om efter 1 augusti, börja från i år
        end_date = f"{current_year + 1}-07-01"    # Slutdatum är 1 juli nästa år

    # Konvertera start- och slutdatum till datetime-objekt
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Filtrera ut matcher för den valda ligan och inom det specifika datumintervallet
    matches_df = df[(df['Div'] == league) & 
                    (pd.to_datetime(df['Date']) >= start_date) & 
                    (pd.to_datetime(df['Date']) < end_date)]
    
    # Initialize a dictionary to store statistics for each team
    league_table = {}

    for index, match in matches_df.iterrows():
        home_team = match['HomeTeam']
        away_team = match['AwayTeam']
        home_goals = match['FTHG']
        away_goals = match['FTAG']

        # Initialize teams in the league table if not already present
        if home_team not in league_table:
            league_table[home_team] = {'Played': 0, 'Won': 0, 'Drawn': 0, 'Lost': 0,
                                       'Goals For': 0, 'Goals Against': 0, 'Goal Difference': 0, 'Points': 0}
        if away_team not in league_table:
            league_table[away_team] = {'Played': 0, 'Won': 0, 'Drawn': 0, 'Lost': 0,
                                       'Goals For': 0, 'Goals Against': 0, 'Goal Difference': 0, 'Points': 0}

        # Update statistics for the home team
        league_table[home_team]['Played'] += 1
        league_table[home_team]['Goals For'] += home_goals
        league_table[home_team]['Goals Against'] += away_goals
        league_table[home_team]['Goal Difference'] += (home_goals - away_goals)

        # Update statistics for the away team
        league_table[away_team]['Played'] += 1
        league_table[away_team]['Goals For'] += away_goals
        league_table[away_team]['Goals Against'] += home_goals
        league_table[away_team]['Goal Difference'] += (away_goals - home_goals)

        # Determine the result and update points
        if home_goals > away_goals:
            league_table[home_team]['Won'] += 1
            league_table[home_team]['Points'] += 3
            league_table[away_team]['Lost'] += 1
        elif home_goals < away_goals:
            league_table[away_team]['Won'] += 1
            league_table[away_team]['Points'] += 3
            league_table[home_team]['Lost'] += 1
        else:
            league_table[home_team]['Drawn'] += 1
            league_table[away_team]['Drawn'] += 1
            league_table[home_team]['Points'] += 1
            league_table[away_team]['Points'] += 1

    # Convert the dictionary to a DataFrame
    league_df = pd.DataFrame.from_dict(league_table, orient='index')

    # Sort the DataFrame by Points, Goal Difference, and Goals For
    league_df = league_df.sort_values(by=['Points', 'Goal Difference', 'Goals For'], ascending=False)

    return league_df